In [1]:
library("readr")
library("tximport")
library("tximportData")
library("DESeq2")
# Summarized - Experiment 

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: 'S4Vectors'


The following objects are masked from 'package:base':

    expand.grid, I, unname


Loading required package: IRanges


Attaching package: 'IRanges'


The following object is masked from 'package:grDevices':

    windows


Loading required package: GenomicRanges

Loading required p

## condition - Joined Dataset

 In order to compare between LR ans NR, need to remove all "Contol" samples, which are not used during this comparison. 
 so each "healthy"/CTRL samples are removed

In [2]:

dir <- "C:/python/projects/bioinformatics/rnaseq/data/dataset_joined"

# readt count matrix (non-normalized)
count_mat_fname <- file.path(dir, 'countMatrix_include_gene_name.csv')
cts <- as.matrix(read.csv(count_mat_fname,sep=",",row.names="gene_name"))

# read samples meta-data
coldata_fname <- file.path(dir, 'samples_joined.csv')
## read sample table and set condition as factor
cc <- c(condition = "factor", batch = "factor") # a named vector of column classes

coldata <- read.csv(coldata_fname,sep=",", colClasses = cc ,row.name = 1)

# drop healthy samples
coldata <- coldata[coldata$diagnosis == 'BD',]


In [3]:
# sort by condition
coldata<- coldata[order(coldata$condition),]

## arraange countmatrix  according columns order in the coldata
cts <- cts[,rownames(coldata)]

In [4]:
# Load data into deseq2 dataframe
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~  batch + condition)

factor levels were dropped which had no samples



In [5]:
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 167 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [6]:
resultsNames(dds)

[1] "Intercept"          "batch_2_vs_1"       "batch_3_vs_1"      
[4] "batch_4_vs_1"       "condition_NR_vs_LR"

In [7]:
dds

class: DESeqDataSet 
dim: 39397 56 
metadata(1): version
assays(6): counts mu ... replaceCounts replaceCooks
rownames(39397): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(35): baseMean baseVar ... maxCooks replace
colnames(56): X_R9288_1 X_Rou7776B_1 ... X_R29425B_1 X_523_1
colData names(8): batch diagnosis ... sizeFactor replaceable

In [8]:
res <- results(dds, contrast=c("condition","NR","LR"))

In [9]:
summary(res)


out of 32253 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 49, 0.15%
LFC < 0 (down)     : 17, 0.053%
outliers [1]       : 94, 0.29%
low counts [2]     : 13575, 42%
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [10]:
saveRDS(dds,'dds_batch_condition_4batches_joined.rds')

## condition - Original Dataset

In [11]:

dir <- "C:/python/projects/bioinformatics/rnaseq/data/dataset_joined"

# readt count matrix (non-normalized)
count_mat_fname <- file.path(dir, 'countMatrix_include_gene_name.csv')
cts <- as.matrix(read.csv(count_mat_fname,sep=",",row.names="gene_name"))

# read samples meta-data
coldata_fname <- file.path(dir, 'samples_original.csv')
## read sample table and set condition as factor
cc <- c(condition = "factor", batch = "factor") # a named vector of column classes

coldata <- read.csv(coldata_fname,sep=",", colClasses = cc ,row.name = 1)

# drop healthy samples
coldata <- coldata[coldata$diagnosis == 'BD',]

In [12]:
# sort by condition
coldata<- coldata[order(coldata$condition),]

## arraange countmatrix  according columns order in the coldata
cts <- cts[,rownames(coldata)]

In [13]:
# Load data into deseq2 dataframe
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~  batch + condition)

factor levels were dropped which had no samples



In [14]:
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 206 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [15]:
resultsNames(dds)

[1] "Intercept"          "batch_2_vs_1"       "batch_3_vs_1"      
[4] "condition_NR_vs_LR"

In [16]:
dds

class: DESeqDataSet 
dim: 39397 32 
metadata(1): version
assays(6): counts mu ... replaceCounts replaceCooks
rownames(39397): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(31): baseMean baseVar ... maxCooks replace
colnames(32): X_R9288_1 X_Rou7776B_1 ... X_R25549B_1 X_R29425B_1
colData names(8): batch diagnosis ... sizeFactor replaceable

In [17]:
res <- results(dds, contrast=c("condition","NR","LR"))

In [18]:
summary(res)


out of 30144 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 140, 0.46%
LFC < 0 (down)     : 83, 0.28%
outliers [1]       : 167, 0.55%
low counts [2]     : 10906, 36%
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [19]:
saveRDS(dds,'dds_batch_condition_3batches_original.rds')

## diagnosis - Original dataset Only

In [20]:

dir <- "C:/python/projects/bioinformatics/rnaseq/data/dataset_joined"

# readt count matrix (non-normalized)
count_mat_fname <- file.path(dir, 'countMatrix_include_gene_name.csv')
cts <- as.matrix(read.csv(count_mat_fname,sep=",",row.names="gene_name"))

# read samples meta-data
coldata_fname <- file.path(dir, 'samples_original.csv')
## read sample table and set condition as factor
cc <- c(condition = "factor", batch = "factor", diagnosis="factor") # a named vector of column classes

coldata <- read.csv(coldata_fname,sep=",", colClasses = cc ,row.name = 1)

# sort by condition
coldata<- coldata[order(coldata$condition),]
## arraange countmatrix  according columns order in the coldata
cts <- cts[,rownames(coldata)]

In [21]:
#coldata

In [22]:
# Load data into deseq2 dataframe
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ batch + diagnosis)

In [23]:
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 127 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [24]:
saveRDS(dds,'dds_batch_diagnosis_original.rds')